# Database from
https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data?select=full_df.csv

In [1]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from imblearn.under_sampling import RandomUnderSampler

from helpers.help import *
from helpers.helptf import *
from sklearn.utils import resample

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input

# Getting rethinophaty data

In [2]:
# load dataset
df = pd.read_csv('hr-dataset/full_df.csv')

#Left eye
# get the diagnostic of hypertensive retinopathy
ds_hr_left = df[df['Left-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_left = df[df['Left-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic of normal fundus
ds_normal_left = df[df['Left-Diagnostic Keywords'] == 'normal fundus']


#Right eye
# get the diagnostic of hypertensive retinopathy
ds_hr_right = df[df['Right-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_right = df[df['Right-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic of normal fundus
ds_normal_right = df[df['Right-Diagnostic Keywords'] == 'normal fundus']


# Specific dataframe
# Left eye
df_hr_left = ds_hr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_dr_left = ds_dr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_normal_left = ds_normal_left[['Left-Diagnostic Keywords', 'Left-Fundus']]

# Right eye
df_hr_right = ds_hr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_dr_right = ds_dr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_normal_right = ds_normal_right[['Right-Diagnostic Keywords', 'Right-Fundus']]


# Droping class
# Left eye
df_hr_left = df_hr_left.drop('Left-Diagnostic Keywords', axis=1)
df_dr_left = df_dr_left.drop('Left-Diagnostic Keywords', axis=1)
df_normal_left = df_normal_left.drop('Left-Diagnostic Keywords', axis=1)
# Right eye
df_hr_right = df_hr_right.drop('Right-Diagnostic Keywords', axis=1)
df_dr_right = df_dr_right.drop('Right-Diagnostic Keywords', axis=1)
df_normal_right = df_normal_right.drop('Right-Diagnostic Keywords', axis=1)



print("---- LEFT ----")
print(df_hr_left.shape[0])
print(df_dr_left.shape[0])
print(df_normal_left.shape[0])
print("---- RIGHT ----")
print(df_hr_right.shape[0])
print(df_dr_right.shape[0])
print(df_normal_right.shape[0])


---- LEFT ----
191
85
2796
---- RIGHT ----
191
80
2705


# Solving the undersampling of HR 

In [3]:
# join dataframes
df_hr = pd.concat([df_hr_left, df_hr_right])
df_dr = pd.concat([df_dr_left, df_dr_right])
df_normal = pd.concat([df_normal_left, df_normal_right])

df_hr_downsampled = resample(df_hr, replace=False, n_samples=200, random_state=10)
df_dr_downsampled = resample(df_hr, replace=False, n_samples=165, random_state=10)
df_normal_downsampled = resample(df_normal, replace=False, n_samples=220, random_state=10)

print(df_hr_downsampled.shape[0])
print(df_dr_downsampled.shape[0])
print(df_normal_downsampled.shape[0])

print(df_hr_downsampled)


200
165
220
        Left-Fundus    Right-Fundus
2665            NaN  4077_right.jpg
5899  4122_left.jpg             NaN
688             NaN   790_right.jpg
1350            NaN  1989_right.jpg
3685   571_left.jpg             NaN
...             ...             ...
5752  3951_left.jpg             NaN
3348            NaN   176_right.jpg
3350            NaN   179_right.jpg
5906  4129_left.jpg             NaN
2772  4202_left.jpg             NaN

[200 rows x 2 columns]


# Class transformation

In [4]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'hr-dataset/preprocessed_images')

# 0 - Normal
# 1 - Diabetic Rethinopaty
# 2 - Hipertensive Rethinopaty

# roam Hipertensive rethinopaty
array = []
detailPath = ""
for index, row in df_hr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,2])


# roam Diabeic rethinopaty
for index, row in df_dr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,1])

# roam no rethinopaty
for index, row in df_dr_downsampled.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        array.append([detailPath,0])


    
# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

530

# Get the data ready

In [5]:
X,y=dataset[::,0],dataset[::,1]
y = y.astype(int)

#One hot encode the labels
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 20% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.2*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 20% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.18, random_state=42)

In [6]:
print(f"Samples in Training set: {x_train.shape[0]}")
print(f"Samples in Validation set: {x_val.shape[0]}")
print(f"Samples in Test set: {x_test.shape[0]}")

Samples in Training set: 347
Samples in Validation set: 77
Samples in Test set: 106


In [7]:
# Check if imbalance
for i in [y_train, y_test, y_val]:
    print(np.unique(i, return_counts = True, axis = 0))

(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([126, 113, 108]))
(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([43, 29, 34]))
(array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([31, 23, 23]))


# Prepares Data for the model

In [8]:
val_dataset=build_dataset(x_val,y_val,repeat=False,batch=32)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=32)

BATCH_SIZE=32
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_train,y_train,batch=BATCH_SIZE)

# input shape for the model
input_shape=train_dataset.element_spec[0].shape[1:]


print(train_dataset)
print(val_dataset)
print(test_dataset)

input_shape=train_dataset.element_spec[0].shape[1:]
print(input_shape)

for batch in train_dataset.take(1):
    features, labels = batch  # Unpack the tuple
    print(features.shape[0])  # Number of elements in the batch
    print(labels.shape[0])  # Number of elements in the batch

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
(224, 224, 3)
32
32


2024-11-25 00:46:39.041272: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Load model

In [ ]:
dr_model = load_model('model/model_al.keras')

"""
print(dr_model.layers)

# Define the input tensor explicitly
input_tensor = Input(shape=input_shape)  # Match the original input shape

# Create the feature extractor
feature_extractor = Model(inputs=input_tensor, outputs=dr_model.layers[-7].output)


for layer in feature_extractor.layers:
    layer.trainable = False  # Freeze feature extractor

print(input_shape)
print(dr_model.layers)
"""
dr_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 13, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,053,192 (7.83 MB)

 Trainable params: 684,162 (2.61 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 1,368,326 (5.22 MB)

# Add the new layer

In [14]:
# classes: {"no_rethinopathy", "dr","hr"}
transfer_model = prep_translearn(model=dr_model, top_layers_to_cut=6, out_dim=3, learning_rate=0.001) 

After layer 0 (conv2d_3), shape: (None, 224, 224, 32)
After layer 1 (batch_normalization_4), shape: (None, 224, 224, 32)
After layer 2 (max_pooling2d_3), shape: (None, 112, 112, 32)
After layer 3 (dropout_4), shape: (None, 112, 112, 32)
After layer 4 (conv2d_4), shape: (None, 110, 110, 64)
After layer 5 (batch_normalization_5), shape: (None, 110, 110, 64)
After layer 6 (max_pooling2d_4), shape: (None, 55, 55, 64)
After layer 7 (dropout_5), shape: (None, 55, 55, 64)
After layer 8 (conv2d_5), shape: (None, 53, 53, 128)
After layer 9 (batch_normalization_6), shape: (None, 53, 53, 128)
After layer 10 (max_pooling2d_5), shape: (None, 26, 26, 128)


# Prepare transfer model

In [15]:
transfer_model.compile(
        loss = "categorical_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )

transfer_model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ NEW_input (InputLayer)          │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 110, 110, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 110, 110, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 53, 53, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 53, 53, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_FLAT (Flatten)              │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_Signature (Dense)           │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NEW_output (Dense)              │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,243 (42.61 MB)

 Trainable params: 11,169,795 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

# Train model

In [16]:
# saves the model with the lowest validation Loss
checkpoint=ModelCheckpoint(filepath='model/model_transferlearning.keras',
                           monitor='val_loss',save_best_only=True,verbose=1)

# logs the training progress to a CSV
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_transferlearning.csv',
                                     separator=',',append=False)

# defines a early stop if in 10 epoches the validation loss dont improve
early_stopper=keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0.001,
                                            restore_best_weights=True,
                                            patience=10)

callbacks_list=[checkpoint,early_stopper,csv_logger]

In [17]:
# EPOCHS = 20 # minimalist
# EPOCHS = 200 # standard
EPOCHS = 100 # standard

STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

transfer_model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=EPOCHS,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.3280 - loss: 59.2432
Epoch 1: val_loss improved from inf to 28.10709, saving model to model/model_transferlearning.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - categorical_accuracy: 0.3252 - loss: 58.7946 - val_categorical_accuracy: 0.3766 - val_loss: 28.1071
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.3496 - loss: 16.5467
Epoch 2: val_loss improved from 28.10709 to 20.70378, saving model to model/model_transferlearning.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - categorical_accuracy: 0.3491 - loss: 16.4474 - val_categorical_accuracy: 0.3896 - val_loss: 20.7038
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.3845 - loss: 8.3792
Epoch 3: val_loss improved from 20.70378 to 11.02323, saving model to model/model_transferlearning.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - categorical_accuracy: 0.3831 - loss: 8.3670 - val_categorical_accuracy: 0.428

KeyboardInterrupt: 

# Evaluation

In [ ]:
# load the best model, trained before
model = keras.models.load_model("model/model_transferlearning.keras")
print("-" * 100)

# evaluates with the test_dataset
print(model.evaluate(test_dataset, verbose=0,return_dict=True))

----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.16326530277729034, 'loss': 1.8221027851104736}
